### Predicting Consequences of Over prescription/Overdose Epidemics

### DATA SCIENCE IMMERSIVE FINAL CAPE STONE

This capestone was initaited with an idea that why are so much more physicians presecribing medications that can lead to overdose. I have worked with patients for more than a year in a clinic as well as a hospital and saw significant amount of patients requesting pain relief medications again and again. I have seen people i know been afffected with the chronic and life threatening overdose.

The objective of this capestone is to develop models to identify types of prescribers that are a high-risk for opioid related fatalities and predict most influential opioids leading to deaths due to overdose.

The public health data was extracted from https://wonder.cdc.gov/. I downloaded into individual tab separated text files as dowloading the 3.2GB data at once and extracting needs significant processing power besides I noticed some anomalies when extracting data from Wonder in large due to the system or functionality of wonder tool they have to group the needed data. The otherData was extraced from CMS Medicare Part D Opioid Prescriber Data
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/OpioidMap.html

**Author : Kiros Gebremariam**

Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

Loadding Necessary Libraries

In [1]:
import os                                                                    # os library
import numpy as np                                                           # numpy library
import pandas as pd                                                          # pandas library
import matplotlib.pylab as plt                                               # plotting and visualization library
import seaborn as sns                                                        # plotting
import statsmodels.api as sm                                                 # for running regressions
np.random.seed(2018)                                                         # Random seed for replication 



# Configure visual settings:
plt.style.use('fivethirtyeight')
sns.set(font_scale=2)
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import os                      # just ensuring the files using as working Directory(pwd)
print(os.listdir())

['.DS_Store', '.gitattributes', '.ipynb_checkpoints', '1.1. Capestone Data setup.ipynb', '1.1. Capestone Data setup_1.ipynb', 'Data Files', 'KFFMentalHealthspendState.csv', 'Medicare_Part_D_Opioid_Prescriber_Summary_File_2016.csv', 'opioids.csv', 'overdoses.csv', 'Physician_Locator.csv', 'pop-urban-pct-historical_cleaned_up.csv', 'prescriber-info.csv', 'Prescriber_county.csv', 'Prescriber_state.csv', 'Prescriber_zip.csv', 'PropertyCrimeState1999.csv', 'PropertyCrimeStates2014.csv', 'raw_data_kff mental.csv', 'raw_data_KFF_opioid_death.csv', 'rawdataKFFopioiddeath.csv', 'README.md', 'Statistically-significant-drug-overdose-death-rate-increase-from-2015-to-2016-US-states.csv', 'ViolentCrimeState1999.csv', 'ViolentCrimeState2014.csv', 'Wonder_Drug_Death_Data_1.csv', 'Wonder_Drug_Death_Data_2.csv']


In [3]:
df1 = pd.read_csv('Medicare_Part_D_Opioid_Prescriber_Summary_File_2016.csv')
df2 =pd.read_csv('Physician_Locator.csv')
df2 =pd.read_csv('Prescriber_state.csv')
df3 =pd.read_csv('Prescriber_county.csv')
df4 =pd.read_csv('Prescriber_zip.csv')
df5 =pd.read_csv('PropertycrimeState1999.csv')
df6 =pd.read_csv('PropertyCrimeStates2014.csv')
df7 =pd.read_csv('Physician_Locator.csv')
df8 =pd.read_csv('KFFMentalHealthspendState.csv')
df9 =pd.read_csv('pop-urban-pct-historical_cleaned_up.csv')
df10 = pd.read_csv('rawdataKFFopioiddeath.csv', engine ="python")
df11 = pd.read_csv('ViolentCrimeState1999.csv')
df12 = pd.read_csv('ViolentCrimeState2014.csv')
df13 = pd.read_csv('Wonder_Drug_Death_Data_1.csv')
df14 = pd.read_csv('Wonder_Drug_Death_Data_2.csv')
df15 = pd.read_csv('opioids.csv')
df16 = pd.read_csv('overdoses.csv')
df17 = pd.read_csv('prescriber-info.csv')

# Checking Data Integrity

In [4]:
df1.head()[:2]

,NPI,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Specialty Description,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate,Extended-Release Opioid Claims,Extended-Release Opioid Prescribing Rate
0,1003000126,ENKESHAFI,ARDALAN,21502.0,MD,Internal Medicine,545,23.0,4.22%,NaN,NaN
1,1003000142,KHALIL,RASHID,43623.0,OH,Anesthesiology,1733,1004.0,57.93%,63.0,6.27%


In [5]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df1.isnull().sum()/df1.shape[0]).sort_values(ascending=False)  

Extended-Release Opioid Prescribing Rate    0.656609
Opioid Prescribing Rate                     0.281917
Opioid Claim Count                          0.281917
Extended-Release Opioid Claims              0.113573
NPPES Provider ZIP Code                     0.000047
NPPES Provider First Name                   0.000028
NPPES Provider Last Name                    0.000028
Total Claim Count                           0.000000
Specialty Description                       0.000000
NPPES Provider State                        0.000000
NPI                                         0.000000
dtype: float64

In [6]:
df2.head()[:2]

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2016_Part D Opioid Prescribers,2016_Opioid Claims,2016_Extended Release Opioid Claims,2016_Overall_Claims,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,496350,77716608,5381275,1452978228,5.34,6.92,-0.47,0.80,-0.30,0.59
1,Alabama,AL,1.0,"12,820","7,422","2,260,284","118,523","29,160,952",7.75,5.24,...,"7,004","2,156,504","120,579","28,882,148",7.47,5.59,-0.28,0.35,-0.21,0.31


In [7]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df2.isnull().sum()/df2.shape[0]).sort_values(ascending=False)  

State_Abbreviation                                               0.019231
State_FIPS                                                       0.019231
2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.000000
2013_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2014_Overall_Claims                                              0.000000
2014_Extended_Release_Opioid_Claims                              0.000000
2014_Opioid_Claims                                               0.000000
2014_Part D_Opioid_Prescribers                                   0.000000
2014_Part_ D_Prescribers                                         0.000000
2013_Overall_Claims                                              0.000000
2013_Opioid_Prescribing_Rate                                     0.000000
2014_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2013_Extended_Release_Opioid_Claims                              0.000000
2013_Opioid Claims                    

In [8]:
df3.head()[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,Autauga,1001.0,67,44,"20,482",979,"208,936",9.80,...,5.85,68,43,"15,998","1,159","198,347",8.07,7.24,-1.73,2.46


In [9]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df3.isnull().sum()/df3.shape[0]).sort_values(ascending=False)  

2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.004537
2013_2015_Change_in_Opioid_Prescribing_Rate                      0.004537
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.002592
2015_Opioid_Prescribing_Rate                                     0.002592
2015_Overall_Claims                                              0.002592
2015_Extended Release Opioid Claims                              0.002592
2015_Opioid Claims                                               0.002592
2015_Part D Opioid Prescribers                                   0.002592
2015_Part D Prescribers                                          0.002592
2013_Extended_Release_Opioid_Prescribing_Rate                    0.001944
2013_Part_ D_Prescribers                                         0.001944
2013_Part_Opioid_Prescribers                                     0.001944
2013_Opioid Claims                                               0.001944
2013_Extended_Release_Opioid_Claims   

In [10]:
df4.head()[:2]

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,35004.0,18,9,"1,340",19,"18,632",7.19,1.42,...,6.30,17,7,"1,340",39,"16,830",7.96,2.91,0.77,1.49


In [11]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df4.isnull().sum()/df4.shape[0]).sort_values(ascending=False)  

2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.054068
2013_2015_Change_in_Opioid_Prescribing_Rate                      0.054068
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.029482
2015_Opioid_Prescribing_Rate                                     0.029482
2015_Overall_Claims                                              0.029482
2015_Extended Release Opioid Claims                              0.029482
2015_Opioid Claims                                               0.029482
2015_Part D Opioid Prescribers                                   0.029482
2015_Part D Prescribers                                          0.029482
2014_Extended_Release_Opioid_Claims                              0.028597
2014_Part_ D_Prescribers                                         0.028597
2014_Part D_Opioid_Prescribers                                   0.028597
2014_Opioid_Claims                                               0.028597
2014_Overall_Claims                   

In [12]:
df5.head()[:2]

,State,Population,Property crime rate,Burglary rate,Larceny-theft rate,Motor vehicle theft rate
0,Alabama,4369862,3922.3,884.4,2737.3,300.6
1,Alaska,619500,3728.7,611.3,2688.3,429.1


In [13]:
df6.head()[:2]

,State,Population,Property crime rate,Burglary rate,Larceny-theft rate,Motor vehicle theft rate
0,Alabama,4849377,3177.6,819.0,2149.5,209.1
1,Alaska,736732,2760,427.6,2096.4,236.0


In [14]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df4.isnull().sum()/df4.shape[0]).sort_values(ascending=False)  

2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.054068
2013_2015_Change_in_Opioid_Prescribing_Rate                      0.054068
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.029482
2015_Opioid_Prescribing_Rate                                     0.029482
2015_Overall_Claims                                              0.029482
2015_Extended Release Opioid Claims                              0.029482
2015_Opioid Claims                                               0.029482
2015_Part D Opioid Prescribers                                   0.029482
2015_Part D Prescribers                                          0.029482
2014_Extended_Release_Opioid_Claims                              0.028597
2014_Part_ D_Prescribers                                         0.028597
2014_Part D_Opioid_Prescribers                                   0.028597
2014_Opioid_Claims                                               0.028597
2014_Overall_Claims                   

In [15]:
df7.head()[:2]

,Unnamed: 0,First,Last,Deg.,Address,City,County,State,Postal Code,Tele.,Fax
0,NaN,Elizabeth,Alexander,NP,Keystone Substance Abuse Services195 S Herlong...,Rock Hill,NaN,South Carolina,29732,7818255752,NaN
1,NaN,Denise,Bouchard,NP,53 Schoodic Drive,Belfast,Waldo,Maine,04915,2073386900,2073386944


In [16]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df7.isnull().sum()/df7.shape[0]).sort_values(ascending=False)  

Unnamed: 0     0.464177
Fax            0.166965
County         0.143704
Deg.           0.108440
Tele.          0.001789
Postal Code    0.000000
State          0.000000
City           0.000000
Address        0.000000
Last           0.000000
First          0.000000
dtype: float64

In [17]:
df8.head()[:2]

,Location,FY2004__SMHA Expenditures Per Capita,FY2005__SMHA Expenditures Per Capita,FY2006__SMHA Expenditures Per Capita,FY2007__SMHA Expenditures Per Capita,FY2008__SMHA Expenditures Per Capita,FY2009__SMHA Expenditures Per Capita,FY2010__SMHA Expenditures Per Capita,FY2011__SMHA Expenditures Per Capita,FY2012__SMHA Expenditures Per Capita,FY2013__SMHA Expenditures Per Capita,Footnotes
0,United States,93.04,99.55,103.53,113.27,121.13,122.90,120.56,123.93,124.99,119.62,NaN
1,Alabama,58.78,60.31,64.43,74.03,79.39,77.89,78.19,78.33,76.27,72.64,NaN


In [18]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df8.isnull().sum()/df8.shape[0]).sort_values(ascending=False)  

Footnotes                               0.584416
FY2007__SMHA Expenditures Per Capita    0.337662
FY2010__SMHA Expenditures Per Capita    0.324675
FY2009__SMHA Expenditures Per Capita    0.324675
FY2008__SMHA Expenditures Per Capita    0.324675
FY2006__SMHA Expenditures Per Capita    0.324675
FY2005__SMHA Expenditures Per Capita    0.324675
FY2004__SMHA Expenditures Per Capita    0.324675
FY2013__SMHA Expenditures Per Capita    0.311688
FY2012__SMHA Expenditures Per Capita    0.311688
FY2011__SMHA Expenditures Per Capita    0.311688
Location                                0.103896
dtype: float64

In [19]:
df9.head()[:2]

,FIPS,Area Name,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,0,United States,39.6,45.6,51.2,56.1,56.5,64.0,69.9,73.6,73.7,75.2,79.0,80.7
1,1,Alabama,11.9,17.3,21.7,28.1,30.2,43.8,54.8,58.6,60.0,60.4,55.4,59.0


In [20]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df9.isnull().sum()/df9.shape[0]).sort_values(ascending=False)  

2010         0.118644
2000         0.118644
1990         0.118644
1980         0.118644
1970         0.118644
1960         0.118644
1950         0.118644
1940         0.118644
1930         0.118644
1920         0.118644
1910         0.118644
1900         0.118644
Area Name    0.118644
FIPS         0.033898
dtype: float64

In [21]:
df10.head()[:2]

,Location,1999__Opioid Overdose Death Rate (Age-Adjusted),2000__Opioid Overdose Death Rate (Age-Adjusted),2001__Opioid Overdose Death Rate (Age-Adjusted),2002__Opioid Overdose Death Rate (Age-Adjusted),2003__Opioid Overdose Death Rate (Age-Adjusted),2004__Opioid Overdose Death Rate (Age-Adjusted),2005__Opioid Overdose Death Rate (Age-Adjusted),2006__Opioid Overdose Death Rate (Age-Adjusted),2007__Opioid Overdose Death Rate (Age-Adjusted),2008__Opioid Overdose Death Rate (Age-Adjusted),2009__Opioid Overdose Death Rate (Age-Adjusted),2010__Opioid Overdose Death Rate (Age-Adjusted),2011__Opioid Overdose Death Rate (Age-Adjusted),2012__Opioid Overdose Death Rate (Age-Adjusted),2013__Opioid Overdose Death Rate (Age-Adjusted),2014__Opioid Overdose Death Rate (Age-Adjusted),2015__Opioid Overdose Death Rate (Age-Adjusted)
0,United States,2.9,3,3.3,4.1,4.5,4.7,5.1,5.9,6.1,6.4,6.6,6.8,7.3,7.4,7.9,9.0,10.4
1,Alabama,0.8,1,1.3,1.6,1.1,1.8,1.8,2.7,3.6,4.1,4.5,4.1,3.8,3.6,3.5,5.6,6.1


In [22]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df10.isnull().sum()/df10.shape[0]).sort_values(ascending=False)  

2015__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2014__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
1999__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2000__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2001__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2002__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2003__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2004__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2005__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2006__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2007__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2008__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2009__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2010__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2011__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2012__Opioid Overdose Death Rate (Age-Adjusted)    0.223881
2013__Opioid Overdose Death Rate (Age-Ad

In [23]:
df11.head()[:2]

,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5
1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6


In [24]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df11.isnull().sum()/df11.shape[0]).sort_values(ascending=False)  

Revised rape rate /2                         1.000000
Aggravated assault rate                      0.702857
Robbery rate                                 0.702857
Legacy rape rate /1                          0.702857
Murder and nonnegligent manslaughter rate    0.702857
Population                                   0.685714
State                                        0.411429
Violent Crime rate                           0.325714
dtype: float64

In [25]:
df12.head()[:2]

,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4849377,427.4,5.7,29.6,41.3,96.9,283.4
1,Alaska,736732,635.8,5.6,75.3,104.7,85.4,440.2


In [26]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df12.isnull().sum()/df12.shape[0]).sort_values(ascending=False)  

Aggravated assault rate                      0.702857
Robbery rate                                 0.702857
Revised rape rate /2                         0.702857
Legacy rape rate /1                          0.702857
Murder and nonnegligent manslaughter rate    0.702857
Population                                   0.685714
State                                        0.411429
Violent Crime rate                           0.325714
dtype: float64

In [27]:
df13.head()[:2]

,Notes,State,State Code,Year,Year Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,1999.0,1999.0,Female,F,84.0,2293259.0,3.7
1,NaN,Alabama,1.0,1999.0,1999.0,Male,M,111.0,2136882.0,5.2


In [28]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df13.isnull().sum()/df13.shape[0]).sort_values(ascending=False)  

Notes          0.961905
Crude Rate     0.063305
Population     0.038095
Deaths         0.038095
Gender Code    0.038095
Gender         0.038095
Year Code      0.038095
Year           0.038095
State Code     0.038095
State          0.038095
dtype: float64

In [29]:
df14.head()[:2]

,Notes,Year,Year Code,State,State Code,Gender,Gender Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Crude Rate Standard Error,% of Total Deaths
0,Total,1999.0,1999.0,Alabama,1.0,Female,F,NaN,NaN,21.0,2293259.0,0.9,0.2,0.0
1,NaN,1999.0,1999.0,Alabama,1.0,Male,M,25-34 years,25-34,11.0,300073.0,Unreliable,1.1,0.0


In [30]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df14.isnull().sum()/df14.shape[0]).sort_values(ascending=False)  

Notes                        0.694154
Ten-Year Age Groups Code     0.305846
Ten-Year Age Groups          0.305846
Gender Code                  0.111634
Gender                       0.111634
State Code                   0.011175
State                        0.011175
Year Code                    0.009175
Year                         0.009175
% of Total Deaths            0.009058
Crude Rate Standard Error    0.009058
Crude Rate                   0.009058
Population                   0.009058
Deaths                       0.009058
dtype: float64

In [31]:
df15.head()[:2]

,Drug Name,Generic Name
0,ABSTRAL,FENTANYL CITRATE
1,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE


In [32]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df15.isnull().sum()/df16.shape[0]).sort_values(ascending=False)  

Generic Name    0.0
Drug Name       0.0
dtype: float64

In [33]:
df16.head()[:2]

,State,Population,Deaths,Abbrev
0,Alabama,"4,833,722",723,AL
1,Alaska,"735,132",124,AK


In [34]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df16.isnull().sum()/df16.shape[0]).sort_values(ascending=False)  

Abbrev        0.0
Deaths        0.0
Population    0.0
State         0.0
dtype: float64

In [35]:
df17.head()[:2]

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1


In [36]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df17.isnull().sum()/df17.shape[0]).sort_values(ascending=False)  

Credentials                  0.03052
Opioid.Prescriber            0.00000
ENALAPRIL.MALEATE            0.00000
FUROSEMIDE                   0.00000
FLUTICASONE.PROPIONATE       0.00000
FLUOXETINE.HCL               0.00000
FLUCONAZOLE                  0.00000
FLOVENT.HFA                  0.00000
FINASTERIDE                  0.00000
FENTANYL                     0.00000
FENOFIBRATE                  0.00000
FELODIPINE.ER                0.00000
FAMOTIDINE                   0.00000
EXELON                       0.00000
ESTRADIOL                    0.00000
ESCITALOPRAM.OXALATE         0.00000
DULOXETINE.HCL               0.00000
GEMFIBROZIL                  0.00000
DOXYCYCLINE.HYCLATE          0.00000
DOXEPIN.HCL                  0.00000
DOXAZOSIN.MESYLATE           0.00000
DORZOLAMIDE.TIMOLOL          0.00000
DONEPEZIL.HCL                0.00000
DIVALPROEX.SODIUM.ER         0.00000
DIVALPROEX.SODIUM            0.00000
DIPHENOXYLATE.ATROPINE       0.00000
DIOVAN                       0.00000
D

In [37]:
df1.shape,df2.shape,df3.shape,df4.shape,df5.shape,df6.shape,df7.shape,df8.shape

((1131550, 11),
 (52, 35),
 (3086, 27),
 (19198, 26),
 (169, 6),
 (169, 6),
 (26826, 11),
 (77, 12))

In [38]:
df9.shape,df10.shape,df11.shape,df12.shape,df13.shape,df14.shape,df16.shape,df17.shape

((59, 14),
 (67, 18),
 (175, 8),
 (175, 8),
 (1785, 10),
 (8501, 14),
 (50, 4),
 (25000, 256))

In [39]:
df1.columns,

(Index(['NPI', 'NPPES Provider Last Name', 'NPPES Provider First Name',
        'NPPES Provider ZIP Code', 'NPPES Provider State',
        'Specialty Description', 'Total Claim Count', 'Opioid Claim Count',
        'Opioid Prescribing Rate', 'Extended-Release Opioid Claims',
        'Extended-Release Opioid Prescribing Rate'],
       dtype='object'),)

In [40]:
df11.head()[:2]

,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5
1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6


In [41]:
df12.head()[:2]

,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4849377,427.4,5.7,29.6,41.3,96.9,283.4
1,Alaska,736732,635.8,5.6,75.3,104.7,85.4,440.2


In [42]:
# # compiling all the CSV's in to a single pandas DF
COLUMN_NAMES = ['State', 'Population', 'Violent Crime rate', 'Murder and nonnegligent manslaughter rate', 'Legacy rape rate /1', 
                'Revised rape rate /2', 'Robbery rate', 'Aggravated assault rate']
df = pd.DataFrame(columns=COLUMN_NAMES)
print(df.columns)   # checking consistency of column names
df =pd.concat([df12,df11],ignore_index =True)  # put the  DataFrames together
df.head()[:2]

Index(['State', 'Population', 'Violent Crime rate',
       'Murder and nonnegligent manslaughter rate', 'Legacy rape rate /1',
       'Revised rape rate /2', 'Robbery rate', 'Aggravated assault rate'],
      dtype='object')


,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4849377,427.4,5.7,29.6,41.3,96.9,283.4
1,Alaska,736732,635.8,5.6,75.3,104.7,85.4,440.2


In [43]:
df =pd.concat([df12,df11])  # put the  DataFrames together
df.head()[:2]

,State,Population,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate
0,Alabama,4849377,427.4,5.7,29.6,41.3,96.9,283.4
1,Alaska,736732,635.8,5.6,75.3,104.7,85.4,440.2


In [44]:
# checking the null values of the df and sorting them as percentage of the DF shape
(df.isnull().sum()/df.shape[0]).sort_values(ascending=False)  

Revised rape rate /2                         0.851429
Aggravated assault rate                      0.702857
Robbery rate                                 0.702857
Legacy rape rate /1                          0.702857
Murder and nonnegligent manslaughter rate    0.702857
Population                                   0.685714
State                                        0.411429
Violent Crime rate                           0.325714
dtype: float64

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 174
Data columns (total 8 columns):
State                                        206 non-null object
Population                                   110 non-null object
Violent Crime rate                           236 non-null object
Murder and nonnegligent manslaughter rate    104 non-null float64
Legacy rape rate /1                          104 non-null float64
Revised rape rate /2                         52 non-null float64
Robbery rate                                 104 non-null float64
Aggravated assault rate                      104 non-null float64
dtypes: float64(5), object(3)
memory usage: 24.6+ KB
